### Import

In [1]:
import pandas as pd
import numpy as np
import random
import os
from datetime import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


### Fixed Random-Seed

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

### Load Data

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

### 데이터 전처리 1 : 결측값 대체

In [4]:
# train 데이터셋에서 결측값 확인
train_missing_values = train.isnull().sum()
print("Train 데이터셋의 결측값:")
print(train_missing_values[train_missing_values > 0])  # 결측값이 있는 열만 출력

Train 데이터셋의 결측값:
keyword          137675
referral_path    161107
dtype: int64


In [5]:
train['keyword'].fillna(train['keyword'].mode()[0], inplace=True)
train['referral_path'].fillna(train['referral_path'].mode()[0], inplace=True)

In [6]:
test['keyword'].fillna(train['keyword'].mode()[0], inplace=True)
test['referral_path'].fillna(train['referral_path'].mode()[0], inplace=True)

### 데이터 전처리 2: Label 인코딩 및 변수 제거

In [7]:
categorical_features = list(train.dtypes[train.dtypes == "object"].index)

for i in categorical_features:
    count = train[i].nunique()  # 고유값의 개수를 계산
    print(f"{i}: {count}")

sessionID: 252289
userID: 206024
browser: 70
OS: 21
device: 3
continent: 6
subcontinent: 23
country: 205
traffic_source: 168
traffic_medium: 7
keyword: 623
referral_path: 1578


In [8]:
train = train.drop(columns=['sessionID','userID'],axis=1)
test = test.drop(columns=['sessionID','userID'],axis=1)

In [9]:
encoding_target = list(train.dtypes[train.dtypes == "object"].index)


for i in encoding_target:
    le = LabelEncoder()
    le.fit(train[i])
    train[i] = le.transform(train[i])
    
    # test 데이터의 새로운 카테고리에 대해 le.classes_ 배열에 추가
    # test 데이터에 대해서 직접적으로 fit을 수행할 경우 Data Leakage
    for case in np.unique(test[i]):
        if case not in le.classes_: 
            le.classes_ = np.append(le.classes_, case) 
    
    test[i] = le.transform(test[i])

### x와 y 설정

In [10]:
train_x = train.drop(columns=['TARGET'])
train_y = train['TARGET']

test_x = test

### 모델학습 및 예측

In [11]:
lasso = Lasso(alpha=0.1).fit(train_x,train_y)
preds = lasso.predict(test_x)

### Submission

In [12]:
submission = pd.read_csv('./sample_submission.csv')
submission

sessionID  TARGET
0      SESSION_252289       0
1      SESSION_252290       0
2      SESSION_252291       0
3      SESSION_252292       0
4      SESSION_252293       0
...               ...     ...
79781  SESSION_332070       0
79782  SESSION_332071       0
79783  SESSION_332072       0
79784  SESSION_332073       0
79785  SESSION_332074       0

[79786 rows x 2 columns]

In [13]:
submission['TARGET'] = preds
submission

sessionID     TARGET
0      SESSION_252289  22.826651
1      SESSION_252290   1.362503
2      SESSION_252291   3.512813
3      SESSION_252292   4.196410
4      SESSION_252293   5.924896
...               ...        ...
79781  SESSION_332070   1.424369
79782  SESSION_332071   1.439631
79783  SESSION_332072   4.137534
79784  SESSION_332073   4.521409
79785  SESSION_332074   1.391202

[79786 rows x 2 columns]

In [14]:
submission.to_csv('./baseline_submission.csv', index=False)